<a href="https://colab.research.google.com/github/MarthaBBrito/DataScience/blob/main/Detec%C3%A7%C3%A3o_de_Fraudes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detector de Fraudes
Detectar fraudes em uma base de dados geralmente envolve a criação de um modelo de aprendizado de máquina que identifique padrões anômalos ou fraudulentos.

##Bibliotecas necessárias



In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

##Importando os dados

In [5]:
# Carregar os dados
df = pd.read_csv("fraud_data_pt.csv")
# Visualizar as primeiras linhas
print(df.head())
# Verificar informações gerais
print(df.info())
# Resumo estatístico
print(df.describe())

   id_transacao        valor tipo_transacao  idade_conta_dias  \
0             1   469.268090  transferência              1244   
1             2  3010.121431      pagamento              4999   
2             3  1316.745694          saque               163   
3             4   912.942554          saque              1975   
4             5   169.624870          saque              3292   

   transacao_internacional  possui_chargeback  fraude  
0                        0                  0       0  
1                        0                  0       0  
2                        0                  0       0  
3                        0                  0       0  
4                        1                  0       0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_transacao             1000 non-null   int64  
 1   v

##Pré-processamento

Tratar valores ausentes:
* Remova ou substitua valores nulos.
* Codificar variáveis categóricas: Use `pd.get_dummies` ou `LabelEncoder`.
* Escalar os dados: Normalize ou padronize variáveis numéricas.

In [11]:
# Verificar valores nulos
print(df.isnull().sum())

# Preencher valores nulos com a média (exemplo)
df.fillna(df.mean(), inplace=True)

# Codificar variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

# Separar recursos e rótulo
X = df.drop("fraude", axis=1)  # Suponha que "is_fraud" é a coluna alvo
y = df["fraude"]

fraud_indices = df[df["fraude"] == 1].index

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

id_transacao                    0
valor                           0
idade_conta_dias                0
transacao_internacional         0
possui_chargeback               0
fraude                          0
tipo_transacao_pagamento        0
tipo_transacao_saque            0
tipo_transacao_transferência    0
dtype: int64


##Treinando o modelo

Vamos usar um Random Forest, que é robusto para detecção de fraudes:

In [12]:
# Criar o modelo
model = RandomForestClassifier(random_state=42)
# Treinar o modelo
model.fit(X_train, y_train)
# Fazer previsões
y_pred = model.predict(X_test)

##Avaliando a eficácia do modelo
Avalie a eficácia do modelo usando métricas como precisão, recall e F1-score.

In [13]:
# Matriz de confusão
print(confusion_matrix(y_test, y_pred))

# Relatório de classificação
print(classification_report(y_test, y_pred))

[[291   1]
 [  1   7]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       0.88      0.88      0.88         8

    accuracy                           0.99       300
   macro avg       0.94      0.94      0.94       300
weighted avg       0.99      0.99      0.99       300



##Identificando as fraudes detectadas
Supondo que você tenha:

* O conjunto de teste `(X_test)` para os dados de entrada.
* O alvo real `(y_test)` com os rótulos verdadeiros.
* As previsões do modelo `(y_pred)`.

In [15]:
# Adicionar as previsões ao conjunto de teste
X_test["fraude_real"] = y_test  # Rótulo verdadeiro
X_test["fraude_prevista"] = y_pred  # Previsão do modelo

# Filtrar apenas as transações classificadas como fraudulentas
fraudes_detectadas = X_test[X_test["fraude_prevista"] == 1]

# Exibir as fraudes detectadas pelo modelo
print(fraudes_detectadas)


     id_transacao        valor  idade_conta_dias  transacao_internacional  \
370           371   224.816769              1357                        1   
327           328   946.030472              3737                        1   
917           918  7057.047187              2053                        1   
534           535  2662.800530              2391                        1   
867           868   344.544225              4427                        1   
595           596  1271.930187              2455                        1   
526           527   993.819684              4164                        1   
449           450   149.665706              2121                        1   

     possui_chargeback  tipo_transacao_pagamento  tipo_transacao_saque  \
370                  1                     False                 False   
327                  1                     False                  True   
917                  1                      True                 False   
534       

`fraude_real`:

Mostra a verdade do dado (se a transação era ou não fraudulenta).

`fraude_prevista`:

Mostra o que o modelo previu como fraudulento.

`fraudes_detectadas`:

Exibe todas as transações que o modelo classificou como fraudes (independentemente de serem corretas ou não).

#Identificando as fraudes corretas

In [16]:
fraudes_corretas = X_test[(X_test["fraude_prevista"] == 1) & (X_test["fraude_real"] == 1)]
print(fraudes_corretas)

     id_transacao        valor  idade_conta_dias  transacao_internacional  \
370           371   224.816769              1357                        1   
917           918  7057.047187              2053                        1   
534           535  2662.800530              2391                        1   
867           868   344.544225              4427                        1   
595           596  1271.930187              2455                        1   
526           527   993.819684              4164                        1   
449           450   149.665706              2121                        1   

     possui_chargeback  tipo_transacao_pagamento  tipo_transacao_saque  \
370                  1                     False                 False   
917                  1                      True                 False   
534                  1                      True                 False   
867                  1                     False                 False   
595          

##Verificando erros no modelo
1. Fraudes reais que o modelo não detectou (falsos negativos):


In [17]:
fraudes_nao_detectadas = X_test[(X_test["fraude_real"] == 1) & (X_test["fraude_prevista"] == 0)]
print(fraudes_nao_detectadas)

     id_transacao     valor  idade_conta_dias  transacao_internacional  \
916           917  533.6498              1777                        1   

     possui_chargeback  tipo_transacao_pagamento  tipo_transacao_saque  \
916                  1                     False                  True   

     tipo_transacao_transferência  fraude_real  fraude_prevista  
916                         False            1                0  


2. Transações legítimas classificadas como fraudes (falsos positivos):

In [18]:
falsos_positivos = X_test[(X_test["fraude_real"] == 0) & (X_test["fraude_prevista"] == 1)]
print(falsos_positivos)

     id_transacao       valor  idade_conta_dias  transacao_internacional  \
327           328  946.030472              3737                        1   

     possui_chargeback  tipo_transacao_pagamento  tipo_transacao_saque  \
327                  1                     False                  True   

     tipo_transacao_transferência  fraude_real  fraude_prevista  
327                         False            0                1  
